In [ ]:
import fabio
import ImageD11.cImageD11
import numpy as np
from matplotlib import pyplot as plt
from segmentation import LocalBGCorrector, load_test_image
from tqdm.autonotebook import tqdm

%matplotlib ipympl

In [ ]:
image = load_test_image(
    "/data/visitor/es1416/id11/20240430/RAW_DATA/NaCl52_water/NaCl52_water_FF_0/NaCl52_water_FF_0.h5"
).astype(np.float32)
mask = fabio.open("/data/visitor/es1416/id11/20240430/SCRIPTS/james_ball/2025/S3DXRD/frelon_mask.edf").data.astype(
    bool
)
bg = fabio.open(
    "/data/visitor/es1416/id11/20240430/SCRIPTS/james_ball/2025/S3DXRD/frelon_background_es1416.edf"
).data.astype(np.float32)
flat = fabio.open("/data/visitor/es1416/id11/20240430/SCRIPTS/james_ball/2025/S3DXRD/F36_Nov2023.edf").data.astype(
    np.float32
)
dark = fabio.open(
    "/data/visitor/es1416/id11/20240430/SCRIPTS/james_ball/2025/S3DXRD/dark_20240416.edf"
).data.astype(np.float32)

corrfunc = LocalBGCorrector(
    threshold=100,
    smoothsigma=1.0,
    bgc=0.9,
    bg=bg,
    dark=dark,
    flat=flat,
    mask=mask,
    m_offset_thresh=70,
    m_ratio_thresh=150,
)

corrected = corrfunc(image)

fig, axs = plt.subplots(1, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(13, 7))
axs[0].imshow(image, norm="log", vmin=100, vmax=1000, interpolation="nearest")
axs[1].imshow(corrected, norm="log", vmin=100, vmax=1000, interpolation="nearest")
axs[0].set_title("Raw image")
axs[1].set_title("Corrected")
plt.show()

In [ ]:
import h5py

with h5py.File(
    "/data/visitor/es1416/id11/20240430/RAW_DATA/NaCl52_water/NaCl52_water_FF_0/NaCl52_water_FF_0.h5",
    "r",
) as f:
    dset = f.get("/1.1/measurement/frelon3")[:]

In [ ]:
def process_images():
    for image in tqdm(dset[:50], desc="Correcting images"):
        corrected = corrfunc(image.astype(np.float32))


%prun process_images()

In [ ]:
labels = np.empty(image.shape, dtype="i")
wrk = np.empty(image.shape, dtype="b")
npks = ImageD11.cImageD11.localmaxlabel(corrected, labels, wrk)

In [ ]:
blobs = ImageD11.cImageD11.blobproperties(corrected, labels * (corrected > 0), npks, omega=0.0)
ImageD11.cImageD11.blob_moments(blobs)
enoughpx = blobs[:, ImageD11.cImageD11.s_1] >= 3
goodpeaks = blobs[enoughpx]

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, constrained_layout=True, figsize=(13, 7))
axs[0].imshow(image, norm="log", vmin=100, vmax=1000, interpolation="nearest")
axs[1].imshow(corrected, norm="log", vmin=100, vmax=1000, interpolation="nearest")
axs[2].imshow(image, norm="log", vmin=100, vmax=1000, interpolation="nearest")
fc, sc = goodpeaks[:, 23:25].T
axs[2].scatter(fc, sc, c="red", marker="+")
axs[0].set_title("Raw image")
axs[1].set_title("Corrected")
axs[2].set_title("Raw image with peaks")
plt.show()